Função definida pelo/a desenvolvedor/a para realizar a inclusão de novos dados em um arquivo já criado

In [9]:
def adiciona_dados_excel(arquivo_excel, nome_planilha, dados_novos, colunas):
    #Verifica dados já carregados
    df_origem = pd.DataFrame(pd.read_excel(arquivo_excel, sheet_name=nome_planilha), index=None)
    
    #transforma os dados existentes da arquivo em tuplas
    tupla_origem = [tuple(x) for x in df_origem.values]

    #verifica se os dados novos já existem no arquivo. Se existir não exporta, se Não existir exporta para o excel.
    #Inicia o loop
    i = 0 
    dados_carga = []

    while i < len(dados_novos): #Realiza o loop de acordo com o tamanho de dados novos
        if (dados_novos[i] not in tupla_origem): #verifica se cada dado novo NÃO se encontra na tupla com os dados já exportados
            dados_carga.append(dados_novos[i]) #Se não for repetido, é dado novo. Adiciona na tupla de dados para carga
            i = i + 1
        else:
            i = i + 1
            
    #cria o dataframe com os dados para carga
    df_dados_carga = pd.DataFrame(dados_carga)
 
    #Cria o dataframe com os dados que serão ajustados
    #df_dados_antigos = pd.DataFrame(tupla_origem)    

    #Renomeia colunas dos dataframes (novos dados e dados ajustados, para terem o mesmo nome. É obrigatório para junção)
    #df_dados_carga = df_dados_carga.rename(columns = {0:"Nome",1:"Altura"})
    #df_dados_antigos = df_dados_antigos.rename(columns = {0:"Nome",1:"Altura"})

    df_dados_carga = df_dados_carga.rename(columns = colunas)
    #df_dados_antigos = df_dados_antigos.rename(columns = colunas)

    #Cria o dataframa com os novos dados, os dados já existentes e os dados ajustados
    #df_dados_carga = pd.concat([df_dados_antigos,df_dados_carga])

    #Exporta para o Excel
    #df_dados_carga.to_excel(arquivo_excel, sheet_name=nome_planilha,index=False)
    #TESTAR COM O MESMO NOME E ALTURA DIFERENTE, PARA ILUSTRAR QUE ADICIONA NORMALMENTE.
    #argumento Mode. a = append. w = write. 
    #engine="openpyxl". É necessário sempre que for um append
    with pd.ExcelWriter(arquivo_excel,mode="a", engine="openpyxl", if_sheet_exists="overlay") as atualizar:
        df_dados_carga.to_excel(atualizar, sheet_name=nome_planilha,header=None, startrow=atualizar.sheets[nome_planilha].max_row,index=False)
    

Função definida pelo/a desenvolvedor/a para realizar a inclusão de novos dados e alteração de dados existentes em um arquivo já criado

In [57]:
def adiciona_altera_dados_excel(arquivo_excel, nome_planilha, dados_novos, colunas):
    #Verifica dados já carregados
    df_origem = pd.DataFrame(pd.read_excel(arquivo_excel, sheet_name=nome_planilha), index=None)
    
    #transforma os dados existentes da arquivo em tuplas
    tupla_origem = [tuple(x) for x in df_origem.values]

    #cria uma lista dos nomes já existentes, para verificação de ajustes
    lista_nomes_origem = []
    j = 0
    while j < len(tupla_origem):
        lista_nomes_origem.append(tupla_origem[j][0])
        j = j + 1
    
    #cria uma lista dos alturas já existentes, para verificação de ajustes
    lista_alturas_origem = []
    l = 0
    while l < len(tupla_origem):
        lista_alturas_origem.append(tupla_origem[l][1])
        l = l + 1
    
    #verifica se os dados novos já existem no arquivo. Se existir não exporta, se Não existir exporta para o excel.
    #Inicia o loop
    i = 0 
    dados_carga = []

    while i < len(dados_novos): #Realiza o loop de acordo com o tamanho de dados novos
        if (dados_novos[i] not in tupla_origem): #verifica se cada dado novo NÃO se encontra na tupla com os dados já exportados
            if (dados_novos[i][0] in lista_nomes_origem): #verifica se os nomes novos são iguais aos que estão no excel
                print("O nome",dados_novos[i][0],"já existe no Excel. Deseja alterar a altura?") #Se for igual, pergunta sobre a alteração.
                nome_igual = input("Digite S para sim ou N para não.")
                if (nome_igual.upper() == "S"): 
                    k = 0
                    while k < len(lista_nomes_origem): #Localiza na lista nomes origem, o mesmo nome
                        if dados_novos[i][0] == lista_nomes_origem[k]: #Quando encontrar o nome for uma verdade
                            lista_alturas_origem[k] = dados_novos[i][1] #altera o valor antigo, com o valor novo
                        
                        k = k + 1                
                else:
                    i = i + 1
            else:
                dados_carga.append(dados_novos[i]) #Se não for nome repetido, já é sabido que é dado novo. Adiciona na tupla de dados para carga
                i = i + 1
            
        i = i + 1

        
    #cria o dataframe com os dados para carga
    df_dados_carga = pd.DataFrame(dados_carga)

    #cria o dataframe com os ajustes para alterações de dados existentes
    t = 0
    lista_tupla_origem_ajustada = []
    while t < len(lista_nomes_origem): #Percorre as listas com os ajustes que foram solicitados, de acordo com a resposta Sim.
        tupla_origem_ajustada = (lista_nomes_origem[t],lista_alturas_origem[t]) #combina os dados de nome e altura nova
        lista_tupla_origem_ajustada.append(tupla_origem_ajustada) #cria a tupla com dados ajustados
        t = t + 1
    
    #Cria o dataframe com os dados que serão ajustados
    df_dados_ajustados = pd.DataFrame(lista_tupla_origem_ajustada)    

    #Renomeia colunas dos dataframes (novos dados e dados ajustados, para terem o mesmo nome. É obrigatório para junção)
    #df_dados_carga = df_dados_carga.rename(columns = {0:"Nome",1:"Altura"})
    #df_dados_ajustados = df_dados_ajustados.rename(columns = {0:"Nome",1:"Altura"})

    df_dados_carga = df_dados_carga.rename(columns = colunas)
    df_dados_ajustados = df_dados_ajustados.rename(columns = colunas)

    #Cria o dataframa com os novos dados, os dados já existentes e os dados ajustados
    df_dados_carga = pd.concat([df_dados_ajustados,df_dados_carga])

    #Exporta para o Excel
    df_dados_carga.to_excel(arquivo_excel, sheet_name=nome_planilha,index=False)

# EXERCÍCIO 10

Ajuste o cenário explanado no Exercício 9, para que o mesmo, além de criar um arquivo novo do Excel, possa permitir a inclusão de novos registros no arquivo criado.

Os dados dos/as entrevistados/as ficam a seu critério.

In [53]:
#bibliotecas
from pathlib import Path
import pandas as pd

#criando listas
lista_nome = []
lista_idade = []
lista_estilo = []
lista_horas = []

#lista para tupla
lista_tupla_estilo_musical = []

#inserindo dados
resposta = "S"

while resposta.upper() == "S":
    nome = input("Nome:")
    lista_nome.append(nome)
    
    #Validando tipo de dados 
    while True:
        try:
            idade = int(input("Idade:"))
            #Verifica também se a idade está entre 15 e 100 anos
            if not (idade >= 15 and idade <= 100):
                raise ValueError() #Aciona o Except
        except ValueError: #Se o usuário inserir uma string ou um número que esteja fora do intervalo definido, dará erro.
            print("Insira um número entre 15 e 100 anos.")
            continue #continua com o loop
        else: #Se estiver tudo certo, para o loop, adiciona a idade na lista e segue com as perguntas.
            break
    lista_idade.append(idade)
    
    estilo_musical = input("Estilo musical preferido:")
    lista_estilo.append(estilo_musical)
    
    #Validando tipo de dados 
    while True:
        try:
            horas = int(input("Quantas horas por dia passa ouvindo música:"))
            if (horas > 24):
                raise ValueError()
        except ValueError:
            print("Insira um número menor ou igual a 24.")
            continue
        else:
            break
    lista_horas.append(horas)
    
    #Função de validação da resposta, para garantir que o usuário digite somente S ou N.
    def valida_resposta(arg_resposta):
        if (arg_resposta != "S" and arg_resposta != "N"):
            print("")
            print("Você digitou",arg_resposta,"! Favor digitar S para sim ou N para não.")
            
    execucao = True
    while execucao:
        resposta = input("Deseja inserir uma novo registro? (S para sim. N para não.)").upper()
        valida_resposta(resposta)
        if (resposta == "S" or resposta == "N"):
            execucao = False    
    
else:
    #criando a tupla
    for i in range(len(lista_nome)):
        dados_estilo_musical = (lista_nome[i],lista_idade[i],lista_estilo[i],lista_horas[i])
        lista_tupla_estilo_musical.append(dados_estilo_musical)
        i = i + 1

        
#exportando para o Excel
#variáveis de sistema de arquivos
endereco_estilo_musical = Path("C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\Senac\\Python - FATEC\\Dados\\")
excel_estilo_musical = endereco_estilo_musical / "estilo_musical.xlsx"

#Criando e atualizando o arquivo Excel
if (endereco_estilo_musical.exists()):
    #Se o endereço existir segue com o processo de criação ou append de dados no excel
    if not (excel_estilo_musical.exists()): #Criar o arquivo pela primeira vez
        #Cria um dataframe com a tupla
        estilo_musical = pd.DataFrame(lista_tupla_estilo_musical)
        
        #Renomeia as colunas
        estilo_musical = estilo_musical.rename(columns = {0:"Nome",1:"Idade",2:"Estilo musical",3:"Horas ouvindo"})
        
        #Envia para o excel
        estilo_musical.to_excel(excel_estilo_musical, sheet_name="estilo_musical",index=False)
        
        print("Arquivo exportado com sucesso!")
    else: #Se o arquivo já existe, somente atualiza os dados      
        #cria um dicionário com o nome das colunas
        colunas = {0:"Nome",1:"Idade",2:"Estilo musical",3:"Horas ouvindo"}
        #aciona a função de append no Excel, enviando o valor da lista
        adiciona_dados_excel (excel_estilo_musical,"estilo_musical",lista_tupla_estilo_musical,colunas)
else:
    #Se o endereço não existir, envia uma mensagem de erro!
    print('Endereço não existe! Favor verificar!')

Nome:Claudio
Idade:42
Estilo musical preferido:Hardcore
Quantas horas por dia passa ouvindo música:5
Deseja inserir uma novo registro? (S para sim. N para não.)N


# EXERCÍCIO 11

Crie um aplicativo que possibilite a entrada de um nome e uma altura (1 por vez), de modo a verificar o seguinte:
- Se o nome já existe no arquivo "alturas.xlsx", deve informar na tela e perguntar se deseja realizar a altaração da altura.
- Se o nome não existir no arquivo "alturas.xlsx", deve inserir o nome e a altura.

In [61]:
#Importando bibliotecas
from pathlib import Path
import pandas as pd

#definindo endereço no sistema de arquivos
endereco = Path("C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\Senac\\Python - FATEC\\Dados\\")

#nome do arquivo
excel = endereco / "alturas.xlsx"

#iniciando a variável
resposta = "S"

#Lógica da verificação dos nomes e alturas
while resposta.upper() == "S":
    #Input dos dados
    while True:
        try:
            nome = input("Nome para ajustar altura:")
            if not (nome.isalpha()):
                raise ValueError()
        except ValueError:
            print("Favor inserir um nome válido")
            continue
        else:
            break
    
    #Input da altura com validação para garantir número decimal      
    while True:
        try:
            altura = float(input("Altura:"))
        except ValueError:
            print("Insira um valor numérico. Ex.: 1.76")
            continue
        else:
            break
            
    #Criação da lista de tupla. Como o requisito é 1 por vez, não há a necessidade de um loop.
    lista_tupla_altura_nova = [] #iniciar a lista
    dados_tupla = (nome,altura) #Cria a tupla
    lista_tupla_altura_nova.append(dados_tupla) #Define a lista com a tupla

    #valida o endereço
    if (endereco.exists()):
        #Dicionário de colunas
        colunas = {0:"Nome",1:"Altura"}
        #Aciona a função que adiciona dados ao Excel
        adiciona_altera_dados_excel(excel,"alturas",lista_tupla_altura_nova, colunas) #argumentos da função
    else:
        print("Endereço não existe!")
    
    #Função de validação da resposta, para garantir que o usuário digite somente S ou N.
    def valida_resposta(arg_resposta):
        if (arg_resposta != "S" and arg_resposta != "N"):
            print("")
            print("Você digitou",arg_resposta,"! Favor digitar S para sim ou N para não.")
            
    execucao = True
    while execucao:
        resposta = input("Processo concluído com sucesso. Deseja ajustar outra altura? (S para sim. N para não.)").upper()
        valida_resposta(resposta)
        if (resposta == "S" or resposta == "N"):
            execucao = False


else:
    print("Aplicativo finalizado.")

Nome para ajustar altura:123
Favor inserir um nome válido
Nome para ajustar altura:Claudio
Altura:1.89
O nome Claudio já existe no Excel. Deseja alterar a altura?
Digite S para sim ou N para não.S
Processo concluído com sucesso. Deseja ajustar outra altura? (S para sim. N para não.)N
Aplicativo finalizado.
